In [ ]:
from nltk.corpus import wordnet as wn

forms = set() #We'll store the derivational forms in a set to eliminate duplicates
for happy_lemma in wn.lemmas(""): #for each "happy" lemma in WordNet
    forms.add(happy_lemma.name()) #add the lemma itself
    for related_lemma in happy_lemma.derivationally_related_forms(): #for each related lemma
        forms.add(related_lemma.name()
        
print(forms))

In [4]:
from neo4j import GraphDatabase

class nGraph():
    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))
    def close(self):
        self._driver.close()
    

    def __run_statement(self, query):
        with self._driver.session() as session:
            return session.write_transaction(self.run, query)

    def run_raw_query(self, query):
        return self.__run_statement(query)

    def get_label(self, entity):
        query = "MATCH (n:ObjectEntity{objectEntity:'"+entity+"'}) RETURN labels(n) as lb"
        r = self.__run_statement(query)[1].records()
        for n in r:
            return n['lb']

    def check_node_exist(self, node_entity):
        cypher_s = "MATCH (n:ObjectEntity{objectEntity:'"+node_entity+"'}) RETURN count(n) <> 0 as res"
        r = self.__run_statement(cypher_s)[1].records()
        for n in r:
            return n['res']
    
    def get_direct_meaning_node(self, entity, r_type):
        query = "MATCH(n:ObjectEntity{objectEntity:'"+entity+"'})-[r:"+r_type+"]-(v:ROOTNODE) return v.objectEntity as root"
        r = self.__run_statement(query)[1].records()
        for n in r:
            return n['root']

    

    def check_no_direct_vn_meaning(self, node_entity):
        cypher_s = "match(n:ObjectEntity{objectEntity:'"+node_entity+"'})-[r:TRANS_EN_VI]-() return count(r) = 0 as res"
        r = self.__run_statement(cypher_s)[1].records()
        for n in r:
            return n['res']

    def get_meaning_of_word(self, w, pos):
        query = "match (n:ObjectEntity{objectEntity:'"+w+"'})-[r:TRANS_EN_VI|:EXPLAINAION_VI_VI]->(v"+pos+")  return n,r,v"
        return self.__run_statement(query)
    def get_meaning_itself(self, w):
        query = "match (n:ObjectEntity{objectEntity:'"+w+"'})-[:LANG_POLY_MEANING]->(v) where exists(v.definition)  return labels(v) as lbs, v.definition as def ,ID(v) as id"
        return self.__run_statement(query)
    def get_en_synonynm(self, _id):
        query = 'match (n)-[:LANG_RELATED_SYNONYM]->(v) where id(n)='+str(_id) + " return v.objectEntity as s"
        return self.__run_statement(query)

    def __getitem__(self, key):
        cypher_s = "MATCH (n {name: '"+key+"'}) return n" 
        return self.__run_statement(cypher_s)
    
    @staticmethod
    def run(tx, message):
        result = tx.run(message)
        avail = result.summary().result_available_after
        cons = result.summary().result_consumed_after
        total_time = avail + cons
        return str(total_time)+" ms", result

In [ ]:
from nltk.stem import PorterStemmer
e_words= ["abolitionizing"]
ps =PorterStemmer()
for w in e_words:
    rootWord=ps.stem(w)
    print(rootWord)

In [ ]:
from nltk import word_tokenize, pos_tag
text = word_tokenize("browner")
pos_tag(text)

In [5]:

g = nGraph("bolt://localhost:7687", "ajax997", "lumia1020")



In [ ]:
results = g.run_raw_query("match (n:ROOTNODE)-[:LANG_GRAMMAR_GERUND]->(v:GERUND) where not (n)-[:LANG_POS_MEANING]-(v) return v.objectEntity as r, labels(v) as lb")[1].records()
for e in results:
    if pos_tag([e['r']])[0][1] == "VBG" and "'" not in e['r']:
        g.run_raw_query("match (n:GERUND{objectEntity:'"+e['r']+"'})-[:LANG_GRAMMAR_GERUND]-(v:ROOTNODE) where not (n)-[:LANG_POS_MEANING]-(v) set n:VBG create (n)-[:LANG_POS_MEANING]->(v)")

        

In [ ]:
from pattern.en import pluralize, singularize

results = g.run_raw_query("match (n:ROOTNODE:ObjectEntity)-[:LANG_POLY_MEANING]->(v:NOUN) where not (n)<-[:LANG_POS_MEANING]-(:NNS) and apoc.text.indexOf(n.objectEntity,' ') = -1 and not n:NNS return n.objectEntity as r")[1].records()

for x in results:
    ori = x['r']
    pr = pluralize(ori).replace("'", "\'")
    if pr[len(pr)-2:] != 'ss':
        
            
        g.run_raw_query("merge (n:NNS{objectEntity:\""+pr+"\", language:'english'}) merge (v:ROOTNODE:ObjectEntity{objectEntity:\""+ori+"\"}) merge (n)-[:LANG_POS_MEANING]->(v)")

            

In [2]:
from pattern.en import pluralize, singularize
print( pluralize('')) #children


penes


In [ ]:
from pattern.en import comparative, superlative
superlative('damp')

In [20]:
results = g.run_raw_query("match (n:ROOTNODE) where not (n)<--(:NNS) and (n)-[:LANG_POLY_MEANING]->(:NOUN) and apoc.text.indexOf(n.objectEntity, ' ')= -1 and right(n.objectEntity, 1)<> 's' return n.objectEntity as r")[1].records()


Failed to read from defunct connection Address(host='localhost', port=7687) (Address(host='::1', port=7687, flow_info=0, scope_id=0))
Transaction failed and will be retried in 1.0809586701801874s (Failed to write to closed connection Address(host='localhost', port=7687) (Address(host='::1', port=7687, flow_info=0, scope_id=0)))


In [21]:
c = 0
for r in results:
    su = pluralize(r['r'])
    
    g.run_raw_query("merge (a:ObjectEntity:ROOTNODE{objectEntity:\""+r['r']+"\"}) merge (b:ObjectEntity:ROOTNODE{objectEntity:\""+su+"\"}) merge(a)<-[:LANG_POS_MEANING]-(b) set b:NNS")
print(c)

0


In [ ]:
from autocorrect import Speller
spell = Speller(lang='en')
spell("sentime")

In [ ]:
literals = "zxcvbnmasdfghjklqpwoeiruty1234567890"
for x in literals:
    g.run_raw_query("create (n:Literal{l:'"+x+"'})")

In [ ]:
"match (l1:Literal{l:'k'}), (l1:Literal{l:'i'}), (l1:Literal{l:'s'}),(l1:Literal{l:'o'}),(l1:Literal{l:'q'}), (v:Root) where (l1)-[r1:PREDICT]->(v) or (l2)-[r2:PREDICT]->(v) or (l3)-[r3:PREDICT]->(v) or (l4)-[r4:PREDICT]->(v) or (l5)-[r5:PREDICT]->(v) with count(r1) + count(r2) + count(r3) + count(r4) + count(r5) as num retrun n orderby num limti 5"

In [ ]:
"optional match (l1:Literal{l:'s'})-[r1:PREDICT]->(v:Root) optional match (l2:Literal{l:'c'})-[r2:PREDICT]->(v:Root) optional match (l3:Literal{l:'a'})-[r3:PREDICT]->(v:Root) optional match(l4:Literal{l:'g'})-[r4:PREDICT]->(v:Root) optional match(l5:Literal{l:'l'})-[r5:PREDICT]->(v:Root) return v, count(r1) + count(r2) + count(r3) + count(r4) + count(r5) as value ORDER BY value DESC limit 10"

In [ ]:
word = "antisematism"
query = ""
return_s = ""
for i in range(0, len(word)):
    return_s += ("+ count(l"+str(i)+")")
    query += "optional match (l1:Literal{l:'"+word[i]+"'})-[r"+str(i)+":PREDICT]->(v:Root) "

print(query +"return " + return_s[1:] + " as value ORDER BY value DESC limit 10")


In [16]:
x = ["furrier",
"flipper",
"advancer",
"salter",
"insider",
"fiver",
"fuller",
"jointer",
"thinner",
"damper",
"ranker",
"matter",
"halter",
"feller",
"goer",
"fancier",
"haler",
"graver",
"faker",
"homer",
"number",
"goner",
"diviner",
"dimmer",
"trimmer",
"liter",
"closer",
"cracker",
"cutter",
"first-rater",
"cleaner",
"flyer",
"freelancer",
"worse",
"fitter",
"rusher",
"cooler",
"diffuser",
"broker",
"flasher",
"wilder",
"welcomer",
"layer",
"wetter",
"mummer",
"part-timer",
"paster",
"owner",
"trigger",
"puffer",
"petter",
"welsher",
"rainier",
"wester",
"patter",
"rasher",
"passer",
"mocker",
"primer",
"meeter",
"minter",
"bluffer",
"bother",
"prompter",
"oliver",
"old-timer",
"bounder",
"popper",
"porter",
"norther",
"bragger",
"boner",
"offer",
"blinder",
"planer",
"muster",
"beater",
"dryer",
"fresher",
"better",
"camper",
"founder",
"loner",
"easter",
"limper",
"buffer",
"leer",
"downer",
"mainer",
"archer",
"bustier",
"buster",
"balder",
"backer",
"liver",
"bummer",
"mariner",
"litter",
"outsider",
"lower",
"idler",
"longer",
"bayer",
"plumber",
"madder",
"loather",
"lighter",
"leaner",
"dresser",
"doubler",
"butcher",
"squinter",
"shammer",
"stapler",
"solder",
"skewer",
"strayer",
"voider",
"tangier",
"roaster",
"stranger",
"serer",
"waster",
"splitter",
"rounder",
"tenner",
"signer",
"sneaker",
"shutter",
"stretcher",
"souther",
"tamer",
"third-rater",
"spreader",
"swaggerer",
"teenager",
"smoother",
"rummer",
"skimmer",
"topper",
"slacker",
"scrubber",
"tanner",
"utterer",
"stocker",
"setter",
"securer",
"steeper",
"snuffer",
"sharper",
"slicker",
"sparer",
"sounder",
"warmer",
"stoner"]



In [17]:
for i in x:
    g.run_raw_query("match (n:ROOTNODE{objectEntity:'"+i+"'})-[r:LANG_POS_MEANING]-(v:NNS) delete r")